In [2]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

In [ ]:
## Boucle cas / Cas par cas 1D

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5
nb_it = 200

# case = 7
# if True :
for case in range(nb_case):
    hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
    folder_path_case = create_folder_paths(folder_path, case, case_name = 'cas_')
    file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = create_save_paths(hyp, folder_path_case, big_it)

    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'

    fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
    fill_temperature(Inputs_PyFluent)

    save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

    save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)

In [ ]:
## Boucle cas / Cas par cas AR

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5
nb_it = 200
limit_big_it = 1
iteration = 0 

c = 0.0001 # m/s

tui, solver = init_solver(folder_path_ansys, server_code)

# case = 7
# if True :
for case in range(nb_case):
    hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
    folder_path_case = create_folder_paths(folder_path, case, case_name = 'cas_')
    file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = create_save_paths(hyp, folder_path_case, big_it)

    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'

    fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
    fill_temperature(Inputs_PyFluent)

    init_ANSYS_parameters(Inputs_PyFluent)

    cy = c/np.sqrt(2)
    cz = - c/np.sqrt(2)
    # L_abs = panelSpecs['main']['L_abs']
    # c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
    jg.standard_initialization(tui, 'cd2_wind', 0, 0, cy, cz)

    while big_it < limit_big_it :

        time_start = time.time()

        solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

        compute_result_CFD(folder_path_case, file_path_result_CFD, file_path_result_PyFluent, panelSpecs, big_it)

        save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

        save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)

        write_files(tui, folder_path_case, case, big_it)

        df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

        fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
        fill_temperature(Inputs_PyFluent)

        if big_it != limit_big_it - 1 :
            change_ANSYS_parameters(tui, Inputs_PyFluent)

        increment_big_it(big_it)

        time_end = time.time()
        Elapsed_time.loc[iteration] = {'iteration': big_it, 'time': time_end - time_start}

    Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case}.csv'),sep=';', index=False)
    
save_sim_1D(Inputs_PyFluent, file_path_result_PyFluent)
    

In [ ]:
## Cas par cas CFD

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
case = 7
nb_it = 200

c = 0.0001 # m/s

tui, solver = init_solver(folder_path_ansys, server_code)

hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
folder_path_case = create_folder_paths(folder_path, case, case_name = 'cas_CFD')
file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = create_save_paths(hyp, folder_path_case, big_it)

df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

hyp['method_h_top_g_exchanger'] = 'CFD'
hyp['method_h_back_abs'] = 'CFD'
hyp['method_h_back_tube'] = 'CFD'
hyp['method_h_rad_back_tube'] = 'CFD'

fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
fill_temperature(Inputs_PyFluent)

init_ANSYS_parameters(Inputs_PyFluent)

cy = c/np.sqrt(2)
cz = - c/np.sqrt(2)
# L_abs = panelSpecs['main']['L_abs']
# c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
jg.standard_initialization(tui, 'cd2_wind', 0, 0, cy, cz)

solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

compute_result_CFD(folder_path_case, file_path_result_CFD, file_path_result_PyFluent, panelSpecs, big_it)

save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)

write_files(tui, folder_path_case, case, big_it)
    

In [ ]:
## Boucle maillage CFD

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')
geometries_dict = pd.read_excel(folder_path + '\\' + 'test-geometries.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
case = 7
nb_geom = len(geometries_dict)
nb_it = 200

c = 0.0001 # m/s

tui, solver = init_solver(folder_path_ansys, server_code)

for geom in range(nb_geom):
    hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
    folder_path_geom = create_folder_paths(folder_path, geom, case_name = 'cas_uniform_M')
    init_mesh(tui, geometries_dict, geom)
    file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = create_save_paths(hyp, folder_path_geom, big_it)

    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'

    fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
    fill_temperature(Inputs_PyFluent)

    init_ANSYS_parameters(Inputs_PyFluent)

    cy = c/np.sqrt(2)
    cz = - c/np.sqrt(2)
    # L_abs = panelSpecs['main']['L_abs']
    # c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
    jg.standard_initialization(tui, 'cd2_wind', 0, 0, cy, cz)

    solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

    compute_result_CFD(folder_path_case, file_path_result_CFD, file_path_result_PyFluent, panelSpecs, big_it)

    save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

    save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)

    write_files(tui, folder_path_case, case, big_it)
    

In [ ]:
## Boucle maillage, cas AR

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')
geometries_dict = pd.read_excel(folder_path + '\\' + 'test-geometries.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5
nb_it = 200
nb_geom = len(geometries_dict)
limit_big_it = 1
iteration = 0 

c = 0.0001 # m/s

tui, solver = init_solver(folder_path_ansys, server_code)
for geom in range(nb_geom) :
    hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
    geom_name = geometries_dict['mesh_name_wo_ext'].iloc[geom]
    folder_path = create_folder_paths(folder_path, geom, case_name = f'cas_{geom_name}')
    init_mesh(tui, geometries_dict, geom)

    end = 0
    T_out_init = 0

    # case = 7
    # if True :
    for case in range(nb_case):
        hyp, big_it, T_amb, T_fluid_in0 = init_hyp(hypotheses_path, stepConditions, case)
        folder_path_case = create_folder_paths(folder_path, case, case_name = 'cas_')
        file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df = create_save_paths(hyp, folder_path_case, big_it)

        df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
        T_out = T_out_init
        T_out_init = df_one['T_fluid_out'].loc[0]

        hyp['method_h_top_g_exchanger'] = 'CFD'
        hyp['method_h_back_abs'] = 'CFD'
        hyp['method_h_back_tube'] = 'CFD'
        hyp['method_h_rad_back_tube'] = 'CFD'

        fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
        fill_temperature(Inputs_PyFluent)

        init_ANSYS_parameters(Inputs_PyFluent)

        cy = c/np.sqrt(2)
        cz = - c/np.sqrt(2)
        # L_abs = panelSpecs['main']['L_abs']
        # c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
        jg.standard_initialization(tui, 'cd2_wind', 0, 0, cy, cz)

        while big_it < limit_big_it :

            time_start = time.time()

            solver.solution.run_calculation.iterate(number_of_iterations=nb_it)

            compute_result_CFD(folder_path_case, file_path_result_CFD, file_path_result_PyFluent, panelSpecs, big_it)

            save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df)

            save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent)

            write_files(tui, folder_path_case, case, big_it)

            try :

                df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
                T_out = df_one['T_fluid_out'].loc[0]

                fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx)
                fill_temperature(Inputs_PyFluent)

                if big_it != limit_big_it - 1 :
                    change_ANSYS_parameters(tui, Inputs_PyFluent)

                increment_big_it(big_it)

            except : 
                big_it = limit_big_it
                continue

            if end == 1 : 
                big_it = limit_big_it

            if abs(T_out-T_out_init)< 0.1 :
                end = 1

            time_end = time.time()
            Elapsed_time.loc[iteration] = {'iteration': big_it, 'time': time_end - time_start}

        Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case}.csv'),sep=';', index=False)
        
    save_sim_1D(Inputs_PyFluent, file_path_result_PyFluent)
    

In [4]:
## Fonctions

def init_hyp(hypotheses_path, stepConditions, case) :
    hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
    big_it = hyp['big_it']
    T_amb = stepConditions[case]['T_amb']
    T_fluid_in0 = stepConditions[case]['T_fluid_in0']
    return hyp, big_it, T_amb, T_fluid_in0

def create_folder_paths(folder_path, case, case_name = 'cas_') : 
    folder_path_geom = os.path.join(folder_path, f'{case_name}{case}')
    if not os.path.exists(folder_path_geom):
        os.makedirs(folder_path_geom)
    return folder_path_geom

def init_mesh(tui, geometries_dict, geom) :
    geom_path = geometries_dict['mesh_path'].iloc[geom]
    geom_name = geometries_dict['mesh_name_wo_ext'].iloc[geom]
    jg.change_mesh(tui, geom_path, geom_name)
    jg.create_radiation(tui, geom_name)

def create_save_paths(hyp, folder_path_case, big_it):
    hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

    file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
    file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
    file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
    file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'

    return file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df

def init_solver(folder_path_ansys, server_code):
    solver_path = folder_path_ansys + '\server\\' + f'server_info-{server_code}.txt'
    solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
    tui = solver.tui
    return tui, solver

def modify_invariable(tui, Inputs_PyFluent):
    theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
    jg.change_gravity(tui, theta)
    jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

def fill_Pyfluent(Inputs_PyFluent, res, T_fluid_in0, nb_hx) :
    apb.set_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in0)
    e_PV = 2.5e-3
    volume_PV = 1134*1708*e_PV*1e-6
    Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
    apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)
    for i in range(1, nb_hx + 1):
        apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
        apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])
    
def init_ANSYS_parameters(Inputs_PyFluent) :
    for i in range(len(Inputs_PyFluent)):
        named_expression = Inputs_PyFluent.iloc[i]['named_expression']
        value = Inputs_PyFluent.iloc[i]['value']
        unit = Inputs_PyFluent.iloc[i]['unit']
        jg.change_named_expression(tui, named_expression, value, unit)
    
def fill_temperature(Inputs_PyFluent) :
    for i in range(2, nb_hx+2) :
        T_fluid_in = res[f'part{i}']['df_one']['T_fluid_in'].loc[0]
        apb.set_value(f'T_fluid_in_{i-1}', 'named_expression', Inputs_PyFluent, T_fluid_in)
    T_fluid_in_man = res[f'part1']['df_one']['T_fluid_mean'].loc[0]
    T_fluid_out_man = res[f'part7']['df_one']['T_fluid_mean'].loc[0]
    apb.set_value(f'T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in_man)
    apb.set_value(f'T_fluid_out_man', 'named_expression', Inputs_PyFluent, T_fluid_out_man)

def save_simu_1D(res, nb_hx, file_path_result_df_one, file_path_result_slices_df) :
        keys = list(res['part1']['df_one'].keys())
        values_dict = {key: [] for key in keys}

        for i in range(1, nb_hx+3):
            part_key = f'part{i}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['df_one'][key].values[0])

        result_1D_df_one = pd.DataFrame(values_dict)

        result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
        result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

        keys = list(res['part1']['slices_df'].keys())
        values_dict = {key: [] for key in keys}

        for part in range(1, nb_hx+3):
            part_key = f'part{part}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['slices_df'][key].values[0])

        result_1D_slices_df = pd.DataFrame(values_dict)
        result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)

def change_ANSYS_parameters(tui, Inputs_PyFluent):
        nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
        for part in range(1, nb_hx+1):
            T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
            a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
            b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
            jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
            jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
            jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
        Heat_Generation_Rate=apb.get_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')
        T_fluid_in_man=apb.get_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, 'T_fluid_in_man',T_fluid_in_man, 'K')
        T_fluid_out_man=apb.get_value('T_fluid_out_man', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, 'T_fluid_out_man', T_fluid_out_man, 'K')

def compute_result_CFD(folder_path_case, file_path_result_CFD, file_path_result_PyFluent, panelSpecs, big_it) :
        jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
        jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

        Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
        Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

        Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
        Qdot_conv['Component'] = Qdot_tot['Component']
        Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

        Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

        hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
            'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

        ## Partie très moche, difficile de faire autrement pour le moment
        part1 = ['manifold_yu']
        part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
        part3 = ['hx_flat_yu_air']
        part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
        part5 = ['hx_flat_yd_air']
        part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
        part7 = ['manifold_yd']

        part1_top = []
        part2_top = []
        part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
        part4_top = []
        part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
        part6_top = []
        part7_top = []


        Areas_top = []
        Areas_back = []
        for part in panelSpecs['decomp'].keys() :
            if part != 'main':
                Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
                Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)

        phi_top_list = []
        phi_tube_list = []
        phi_abs_list = []

        for i in range(1, nb_hx+3) :
            if i == 3 or i == 5 :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
                phi_abs_list.append(1e-6)
            else :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(1e-6)
                phi_abs_list.append(1e-6)

        result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
        result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
        result_CFD['phi_top'] = phi_top_list 
        result_CFD['phi_abs'] = phi_abs_list
        result_CFD['phi_tube'] = phi_tube_list
        
        result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)

def save_Pyfluents(Inputs_PyFluent, file_path_result_PyFluent) :
    Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)

def increment_big_it(big_it) :
        big_it = big_it + 1
        hyp['big_it'] = big_it
        file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
        file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
        file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
        file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'
        return file_path_result_CFD, file_path_result_PyFluent, file_path_result_df_one, file_path_result_slices_df

def write_files(tui, folder_path_case, case, big_it) :
        jg.write_data(tui, folder_path_case, f'cas{case}_it{big_it}')
        jg.write_residuals_file(tui, folder_path_case, f'residuals_cas{case}_it{big_it}')
        jg.write_case(tui, folder_path_case, f'cas{case}_it{big_it}')
        jg.compute_mass_flow_rate(tui, 'face-inlet-under-panel', folder_path_case, f'mass_flow_rate_cas{case}_it{big_it}')
        jg.compute_temp_avg(tui, 'face-oulet-under-panel', folder_path_case, f'temp_outlet_cas{case}_it{big_it}')

In [4]:
folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2)
stepConditions = steadyStateConditions_df.to_dict(orient='records')
Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')
geometries_dict = pd.read_excel(folder_path + '\\' + 'test-geometries.xlsx')
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5

df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[0], hyp)